In [8]:
from collections import Counter
import numpy as np

### HELPERS

In [45]:
def election(rows, threshold):
    
    # VOTE & THRESHOLD FOR DECISION
    votes = []
    
    # LOOP THROUGH EACH ELEMENT
    for row in rows:
        vote = decide_vote(row, threshold)
        votes.append(vote)
            
    return votes

In [46]:
def decide_vote(value, threshold):
    if value > threshold:
        return 1
    elif value < -threshold:
        return 2
    else:
        return 3

### TRIGGER

In [49]:
def classify(params):
    
    # DECONSTRUCT PARAMS
    original = params['predictions']
    label_names = params['label_names']
    threshold = params['threshold']
    
    # CLONE THE ORIGINAL
    dataframe = original.copy()
    
    # LOOP THROUGH EACH COLUMN
    for column in dataframe.columns:
        
        # CALCULATE LOG VALUE BY CURRENT & NEXT
        dataframe[column] = np.log(dataframe[column] / dataframe[column].shift(1))
        
    # DROP ROWS WITH NAN VALUES
    dataframe.dropna(inplace=True)
    
    # EXTRACT FEATUERS & LABELS
    labels = dataframe['label'].to_numpy()
    
    # CLASSIFY EACH LABEL VALUE TO A BUY/SELL/HOLD DECISION
    label_votes = election(labels, threshold)
    
    # CHANGE LABEL STRINGS TO NUMERIC REPRESENTATIONS
    #label_votes = rename_labels(label_votes, label_names)
    
    # REPLACE THE OLD LABELS
    dataframe['label'] = label_votes
    
    return dataframe